### Summarization of All Methods

* RandTridiognalHermitian(n): Produces a random Tridiagonal Hermitian Matrix, with all it's entries to be non-zero. 
* RandomHermitianMatrix(n): Produces a matrix that is square, dense, and harmitian.  
* pureQR(itr, m): Performs the pureQR algorithm on a given matrix with a given number of steps. 
* SimultaneousItr(itr, m): Performs a Simultaneous Iteration on a given matrix for a certain number of iterations. 
* PureQRAuto(m): Performs a Simultaneous Iteration on a given matrix until it's sub-diagonals are small enough for a given tolerance. 
* RecursiveShiftedQR!(m): Preforms a shifted QR Iteration recursively on a given Hermitian Matrix. Each recursive step identifies an Eigenvalue for the given matrix. 

### How To solve the Symmetric Eigenvalue Problem? 

There are a lot of algorithm for it, and some of the ones that are efficient are: **"Shifted QR"** and **"Divide and Conquer by Cuppen"**. These are both recursive. 

However, these algorithms are nonly for Tridiagonal matrix, so before we start using these subroutines, we will need to use the Hessenberg Reduction to reduce any Hermitian Matrices into a Tridiagonal Hermitian Matrix and then use these algorithm to look for the decomposition. 

Especiall when using the "Divide and Conquer by Cuppen" it's advisible to have the basecase of 3 by 3 to be solve by the QR algorithm (or maybe symbolically? Lol that will be kinda crazy) etc.

### Bad News
The shifted-QR algorithm is the best I can do, I can't implement the Cuppen's Divide and Conquer Algorithm without doing significant research into the area. It requires a lot tricks to make it works in a stable way. 


In [2]:
using LinearAlgebra
function RandHermitianMatrix(n)
    M = rand(n, n)
    M = triu(M)
    M = M + M'
    return M
end

function pureQR(itr::Int64, m::AbstractArray{<: Number})
    Ak = copy(m)
    Q = I
    for k = 1: itr
        F = qr(Ak)
        Ak = F.R*F.Q
        Q *= Q*F.Q
    end
    return Ak, Q
end

function SimultaneousItr(itr::Int64, m::AbstractArray{<:Number}) 
    Q = I
    A = copy(m)
    for II = 1: itr
        Z = A*Q
        F = qr(Z)  # Not Modified Gram schimt, will it still work?
        Q = F.Q        
    end
    return Q'*A*Q, Q
end

function RandTridiognalHermitian(n:: Int64)
    Result = zeros(n, n)
    for II = 1: n
        Result[II, II] = rand()
    end
    RandomArray = rand(1, n - 1)
    for JJ = 1: n - 1
        Result[JJ, JJ + 1] = RandomArray[JJ]
        Result[JJ + 1, JJ] = RandomArray[JJ]
    end
    return Result
end

function PureQRAuto(m::AbstractArray{<:Number}; Abstol:: Number=1e-6)
    if m' != m
        throw("Matrix is not hermitial, don't use PureQRAuto.")
    end 
    MaxItr = 1e4; Counter = 0;
    A = copy(m)
    Q = I
    while norm(A - Diagonal(A), Inf) > Abstol && Counter < MaxItr
        Decomp = qr(A)
        Q     *= Q*Decomp.Q
        A      = Decomp.R*Decomp.Q
        Counter += 1
    end
    print("Error infinity norm: ", norm(A - Diagonal(A), Inf))
    return diag(A), Q
end


function RecursiveShiftedQR!(m::AbstractArray{<:Number})
    if size(m, 1) != size(m, 2)
        throw("The shifted QR algorithm is only for square matrices but this is not squared: ")
        display(m)
    end
    if size(m, 1) == 1
        # Base case 
        return
    end
    function SingleShiftedQR!(m::AbstractArray{<: Number}; maxItr::Int64 = 1000)
        A = view(m, :, :)
        width = size(m, 1)
        EigenColumn = -1
        # QAccumulated = I                     # This feature is cancelled in this implementation
        Flag = true
        for II = 1: maxItr
            μ = A[end, end] 
            QRDecomp = qr(A - μ.*I)
            Q, R     = QRDecomp.Q, QRDecomp.R
            A[:, :]  = R*Q + μ.*I
            # QAccumulated = QAccumulated*Q
            for JJ = 1: width - 1
                if  abs(A[JJ + 1, JJ]) ≤ 1e-14
                    EigenColumn = JJ
                    Flag = false
                    @goto OutterLoop
                end
            end 
        end
        @label OutterLoop
        if Flag
            # This really should not happen
            println("Wilskin Shift might have failed to converge.")
        end
        return EigenColumn
    end

    JJ          = SingleShiftedQR!(m)
    AUpper      = view(m, 1: JJ, 1: JJ)
    ALower      = view(m, JJ + 1: size(m, 1),  JJ + 1: size(m, 1))
    RecursiveShiftedQR!(AUpper)  # No return value needed
    RecursiveShiftedQR!(ALower)
    m[:, :] = Diagonal(m)
    return
end


RecursiveShiftedQR! (generic function with 1 method)

In [34]:
M = rand(4, 4)
display(pureQR(10, M)[1])
display(pureQR(40, M)[1])


4×4 Array{Float64,2}:
  2.04018      -0.123925    -0.318286    -0.00543013
  7.93811e-5    0.725978     0.315815    -0.195839
 -1.80078e-10   2.93742e-5  -0.261866    -0.691081
 -3.06615e-12  -5.49906e-8   0.00119709   0.149206

4×4 Array{Float64,2}:
  2.04017      -0.124014     -0.318246     -0.00637333
  2.73976e-18   0.725995      0.316376     -0.194934
 -2.64104e-37   1.20946e-18  -0.259853     -0.692272
 -1.69915e-46  -8.82944e-29   4.65343e-11   0.147184

In [13]:
M    = RandHermitianMatrix(5)
D, V = SimultaneousItr(40, M)

display("SimultaneousItr Eigenvalues: ")
display(diag(D))
display("SimultaneousItr Eigenvectors: ")
display(V)
display("The correct answer from julia: ")
display(eigen(M))


"SimultaneousItr Eigenvalues: "

5-element Array{Float64,1}:
  2.951667707661518
  0.9728869360896275
 -0.8942606625712493
  0.16744509387291998
 -0.12001740029901956

"SimultaneousItr Eigenvectors: "

5×5 LinearAlgebra.QRCompactWYQ{Float64,Array{Float64,2}}:
 -0.190417   0.699457  -0.319444   0.184941    0.581596
 -0.483896  -0.331402  -0.200356   0.776052   -0.116691
 -0.321746   0.418286  -0.361112  -0.225738   -0.734953
 -0.447031   0.299203   0.838742   0.0560479  -0.0633357
 -0.652862  -0.369388  -0.15467   -0.556274    0.32243

"The correct answer from julia: "

Eigen{Float64,Float64,Array{Float64,2},Array{Float64,1}}
values:
5-element Array{Float64,1}:
 -0.8944609609372851
 -0.12001740030132191
  0.16744509387522
  0.9730872344556606
  2.951667707661515
vectors:
5×5 Array{Float64,2}:
 -0.326671   0.581597   -0.184939   -0.696111  -0.190417
 -0.196913  -0.116689   -0.776053    0.333459  -0.483896
 -0.365424  -0.734954    0.225736   -0.414524  -0.321746
  0.835598  -0.0633356  -0.0560481  -0.307873  -0.447031
 -0.150837   0.322429    0.556274    0.37097   -0.652862

In [44]:
TridMatrix = RandTridiognalHermitian(5)
display(TridMatrix)
V, Q = SimultaneousItr(50, TridMatrix)
display(V)
display(eigen(TridMatrix))

5×5 Array{Float64,2}:
 0.874553  0.111751  0.0       0.0       0.0
 0.111751  0.90972   0.766485  0.0       0.0
 0.0       0.766485  0.362628  0.412029  0.0
 0.0       0.0       0.412029  0.833455  0.707387
 0.0       0.0       0.0       0.707387  0.303411

5×5 Array{Float64,2}:
  1.62208      -8.0205e-7    3.33067e-16   5.55112e-16   2.77556e-16
 -8.0205e-7     1.23045      6.77865e-8    0.0           1.11022e-16
 -6.66134e-16   6.77865e-8   0.86063       0.0           1.11022e-16
  2.02963e-16  -8.32667e-17  5.55112e-17  -0.405643      5.55112e-17
 -5.20417e-18  -1.73472e-18  1.82146e-16   1.31839e-16  -0.0237483

Eigen{Float64,Float64,Array{Float64,2},Array{Float64,1}}
values:
5-element Array{Float64,1}:
 -0.4056434362624408
 -0.023748345796204884
  0.8606296313406394
  1.2304491067142749
  1.6220800100758561
vectors:
5×5 Array{Float64,2}:
  0.0321679  -0.0529362   0.977941   -0.17929   0.0874581
 -0.368509    0.425524   -0.121846   -0.570989  0.585027
  0.627706   -0.510508   -0.134777   -0.212786  0.530964
 -0.4849     -0.312865    0.0637674   0.614022  0.534695
  0.48376     0.676479    0.0809525   0.468537  0.286832

In [47]:
TridMatrix = RandTridiognalHermitian(5)
display(TridMatrix)
V, Q = pureQR(50, TridMatrix)
display(V)
display(eigen(TridMatrix))

5×5 Array{Float64,2}:
 0.426829  0.281821   0.0       0.0       0.0
 0.281821  0.0978248  0.498723  0.0       0.0
 0.0       0.498723   0.770544  0.494067  0.0
 0.0       0.0        0.494067  0.813606  0.405266
 0.0       0.0        0.0       0.405266  0.595717

5×5 Array{Float64,2}:
  1.4754       -1.77873e-12   3.29946e-16  -1.1238e-16   -4.08299e-17
 -1.77849e-12   0.841116      1.41126e-11  -8.49139e-17   5.28133e-17
  0.0           1.41124e-11   0.506526     -5.31458e-11  -1.27755e-17
  0.0           0.0          -5.31454e-11  -0.311676      3.78059e-12
  0.0           0.0           0.0           3.78057e-12   0.193153

Eigen{Float64,Float64,Array{Float64,2},Array{Float64,1}}
values:
5-element Array{Float64,1}:
 -0.31167591187641275
  0.19315283824690033
  0.5065264109453169
  0.8411160011836564
  1.4754013195480111
vectors:
5×5 Array{Float64,2}:
 -0.298664  -0.355528   0.8332     -0.293058  0.065535
  0.782642   0.294792   0.235625   -0.430806  0.243837
 -0.473856   0.257251  -0.277735   -0.476467  0.636494
  0.247931  -0.598206  -0.0894304   0.366808  0.661917
 -0.110733   0.602221   0.406355    0.605768  0.304942

In [10]:
TridMatrix = RandTridiognalHermitian(100)
m = TridMatrix
D, Q = PureQRAuto(TridMatrix)
display(D)

100-element Array{Float64,1}:
  2.1542220720583996
  2.0411226916509504
  1.815244823372171
  1.756464501353936
  1.7444948094339803
  1.6950430809568315
  1.667593924551519
  1.6479687168751593
  1.6135627193779745
  1.5902112218348512
  1.5832249864744765
  1.5508234129569132
  1.4899651204135802
  ⋮
  0.18066581433023388
  0.17008663003718064
  0.15711483879191396
 -0.13031871639900122
 -0.11813522528562191
  0.10565700268284195
 -0.07574145735735949
 -0.07200593275778314
  0.032102424845557774
  0.02532465275746051
  0.016728086261432625
  0.0018086929058987848

Error infinity norm: 1.110988203134551e-6

However, the pure QR algorithm is hardly an improvement from the Power iterations. And here, we will present the Invers poewr equivalent to the pure QR algorithm. 

The Shifted QR algorithm. 



In [3]:
TridMatrix = RandTridiognalHermitian(512)
D = copy(TridMatrix)
RecursiveShiftedQR!(D)
display(sort(diag(D)))
display(eigen(TridMatrix).values)

512-element Array{Float64,1}:
 -1.4200714271508423
 -1.2905409417755442
 -1.2551219257672699
 -1.227701950801213
 -1.1784593487371335
 -1.1124916209252338
 -1.090426333257894
 -1.0665899408998483
 -1.0613248952929133
 -1.0608778983546565
 -1.0032314742241955
 -0.9887964626652632
 -0.9852924001520184
  ⋮
  2.035421321940322
  2.0632119290382507
  2.0860607392120487
  2.0964436217143847
  2.1265062646028596
  2.139894371847249
  2.175231854229238
  2.1769797785678713
  2.2228421419903057
  2.2283591792054587
  2.245007230880679
  2.407193379655537

512-element Array{Float64,1}:
 -1.420071427150835
 -1.29054094177555
 -1.2551219257672777
 -1.2277019508012244
 -1.1784593487371544
 -1.1124916209252396
 -1.090426333257901
 -1.0665899408998571
 -1.0613248952929193
 -1.0608778983546623
 -1.0032314742242114
 -0.9887964626652666
 -0.9852924001520244
  ⋮
  2.0354213219403374
  2.0632119290382667
  2.0860607392120425
  2.0964436217143545
  2.1265062646028694
  2.139894371847247
  2.1752318542292377
  2.176979778567887
  2.222842141990304
  2.2283591792054698
  2.245007230880682
  2.407193379655565